# bigram model

> Let's create a simple bigram model for text generation based on the Tiny Shakespear dataset

In [ ]:
#| default_exp bigram

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev; nbdev.nbdev_export()

Add and Initiliaze DVC:

In [ ]:
!pdm add dvc
!pdm run dvc init --subdir

Adding packages to default dependencies: dvc
⠹ Fetching hashes for resolved packages... 1.3.3m
🔒 Lock successful
Changes are written to pdm.lock.
Changes are written to pyproject.toml.
⠦ Fetching hashes for resolved packages...
All packages are synced to date, nothing to do.
Installing the project as an editable package...
  ✔ Update framework-gpt 0.0.1 -> 0.0.1 successful
36m⠇ Updating framework-gpt 0.0.1 -> 0.0.1...
🎉 All complete!
36m⠇ Updating framework-gpt 0.0.1 -> 0.0.1...
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+--------------

We will be using PyTorch for this model:

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

Hyperparameters:

In [ ]:
batch_size = 32 # how many sequences will we process in parallel?
block_size = 8 # what is the maximum context length?
max_iters = 3000
eval_interval = 300
learning_rate = 1e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu' # use cuda if available
eval_iters = 200

torch.manual_seed(1337)

<torch._C.Generator>

Getting the Tiny Shakepeare dataset (a small text file containing all the works of Shakepeare):

In [ ]:
!wget -O /home/nclark/git/framework_gpt/nbs/data/raw/input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

path = '/home/nclark/git/framework_gpt/nbs/data/raw/input.txt'
with open(path, 'r', encoding='utf-8') as f:
    text = f.read()
    

In [ ]:
!pdm run dvc stage add -n raw_data \
    -d 00_bigram_dvc.ipynb \
    -d pyproject.toml \
    -d pdm.lock \
    python 00_bigram_dvc.ipynb

ERROR: Stage 'raw_data' already exists in 'dvc.yaml'. Use '--force' to overwrite.


In [ ]:
!pdm run dvc stage add -f \
    -n raw_data \
    -d 00_bigram_dvc.ipynb \
    -d pyproject.toml \
    -d pdm.lock \
    -d data \
    -o 00_bigram_dvc.ipynb \
    python 00_bigram_dvc.ipynb

ERROR: '/home/nclark/git/framework_gpt/nbs/00_bigram_dvc.ipynb' is specified as an output and as a dependency.


Vocabulary Creation and Encoder:

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] #encoder: string -> list of integers
decode = lambda l: ''.join(itos[i] for i in l) #decoder: list of integers -> string

90/10 split into training and validation sets

In [ ]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

Data Loading:

In [ ]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data =train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x, y

Create an estimate_loss function for evaluation while training:

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Bigram Model:

In [ ]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensors of integers
        logits = self.token_embedding_table(idx) #(B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predicions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

Initialize Model:

In [ ]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

Optimizer and Training Loop:

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    # evaluate the loss on train and val sets every eval_interval
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    #evluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.6742, val loss 4.6920
step 1000: train loss 2.5033, val loss 2.5159
step 2000: train loss 2.4623, val loss 2.4900
step 3000: train loss 2.4474, val loss 2.4964
step 4000: train loss 2.4567, val loss 2.4877
step 5000: train loss 2.4525, val loss 2.4839
step 6000: train loss 2.4619, val loss 2.4895
step 7000: train loss 2.4466, val loss 2.4886
step 8000: train loss 2.4475, val loss 2.4915
step 9000: train loss 2.4525, val loss 2.4962


Generate Text:

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


LUCEThinge Bulkisstone Sche,
The we d,

Anomed weron
Fiurwataltho tororent have t,'ld tyou to ard an; he: gesily he t-hravo,
Ang, heancour inowisseeay.

ARoubesirnd y lorge.
WI mee KERin t.
BI che igserd't or!
Weas bed piris we r n foo Y n h
HARDUn t, ur, Be: cther s: at, I RLISee, yp ming thed wast. shabrwowhee
Mayowirsafod polld ORIINT: handsth hir yot'digr t mo hark!
H:
And, neadit! orowie th s he istos
lof avomase RI'd TE m rur ak-
Wheriret s peco br IN: hwomavir
BAs ifabearthor f ad.
Thirer
